In [62]:
%pip install brian2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.9 MB/s eta 0:00:00


In [63]:
# Importing Packages
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import brian2 as b2

In [47]:
# Load your diabetes dataset
df = pd.read_csv("diabetes_data.csv")

In [12]:
df.columns.nunique()

14

In [57]:
df.drop(columns=['ID', 'No_Pation'], inplace=True)

In [48]:
# Target Class
df['CLASS'].unique()

array(['N', 'N ', 'P', 'Y', 'Y '], dtype=object)

In [49]:
# Encoding Target Class
# Diabetic, Non-Diabetic, and Predicted-Diabetic
df['CLASS'] = df['CLASS'].str.strip().str.upper()
print(df['CLASS'].unique())

['N' 'P' 'Y']


1.   'N': Non-Diabetic
2.   'P': Predicted-Diabetic
3.   'Y': Diabetic



In [50]:
df['CLASS'] = df['CLASS'].map({'N': 0, 'P': 1, 'Y': 2})
unique_class = df['CLASS'].unique()
unique_class

,CLASS
0,0
1,0
2,0
3,0
4,0
...,...
995,2
996,2
997,2
998,2


In [51]:
df['Gender'].unique()

array(['F', 'M', 'f'], dtype=object)

In [52]:
# Remove spaces and convert 'f' to 'F' in the 'Gender' column
df['Gender'] = df['Gender'].str.strip().str.upper()

# Get unique values after processing
unique_gender = df['Gender'].unique()
unique_gender

array(['F', 'M'], dtype=object)

In [53]:
df['Gender'] = df['Gender'].map({'F': 0, 'M': 1})
unique_gender = df['Gender'].unique()
unique_gender

In [45]:
# Check for missing values
df.isnull().sum().sum()

1000

In [54]:
df.tail()

,ID,No_Pation,Gender,AGE,Urea,Cr,HbA1c,Chol,TG,HDL,LDL,VLDL,BMI,CLASS
995,200,454317,1,71,11.0,97,7.0,7.5,1.7,1.2,1.8,0.6,30.0,2
996,671,876534,1,31,3.0,60,12.3,4.1,2.2,0.7,2.4,15.4,37.2,2
997,669,87654,1,30,7.1,81,6.7,4.1,1.1,1.2,2.4,8.1,27.4,2
998,99,24004,1,38,5.8,59,6.7,5.3,2.0,1.6,2.9,14.0,40.5,2
999,248,24054,1,54,5.0,67,6.9,3.8,1.7,1.1,3.0,0.7,33.0,2


In [55]:
# List of numerical columns to scale (excluding target and categorical columns)
numerical_cols = ['AGE', 'Urea', 'Cr', 'HbA1c', 'Chol', 'TG', 'HDL', 'LDL', 'VLDL', 'BMI']

# Initialize the scaler
scaler = MinMaxScaler()

# Apply scaling to the numerical columns
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Check the scaled data
print(df[numerical_cols].head())

        AGE      Urea        Cr     HbA1c      Chol        TG       HDL  \
0  0.508475  0.109375  0.050378  0.264901  0.407767  0.044444  0.226804   
1  0.101695  0.104167  0.070529  0.264901  0.359223  0.081481  0.092784   
2  0.508475  0.109375  0.050378  0.264901  0.407767  0.044444  0.226804   
3  0.508475  0.109375  0.050378  0.264901  0.407767  0.044444  0.226804   
4  0.220339  0.171875  0.050378  0.264901  0.475728  0.051852  0.061856   

        LDL      VLDL       BMI  
0  0.114583  0.011461  0.173913  
1  0.187500  0.014327  0.139130  
2  0.114583  0.011461  0.173913  
3  0.114583  0.011461  0.173913  
4  0.177083  0.008596  0.069565  


In [58]:
df.tail()

,Gender,AGE,Urea,Cr,HbA1c,Chol,TG,HDL,LDL,VLDL,BMI,CLASS
995,1,0.864407,0.273438,0.114610,0.403974,0.728155,0.103704,0.103093,0.156250,0.014327,0.382609,2
996,1,0.186441,0.065104,0.068010,0.754967,0.398058,0.140741,0.051546,0.218750,0.438395,0.633043,2
997,1,0.169492,0.171875,0.094458,0.384106,0.398058,0.059259,0.103093,0.218750,0.229226,0.292174,2
998,1,0.305085,0.138021,0.066751,0.384106,0.514563,0.125926,0.144330,0.270833,0.398281,0.747826,2
999,1,0.576271,0.117188,0.076826,0.397351,0.368932,0.103704,0.092784,0.281250,0.017192,0.486957,2


In [59]:
# Preprocess the data
def preprocess_data(df):
    X = df.iloc[:, 1:-1].values  # Features
    y = df.iloc[:, -1].values  # Target

    # Split dataset into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = preprocess_data(df)

In [60]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix

# Define the FNN model
def create_fnn(input_dim):
    model = Sequential()
    model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(3, activation='softmax'))  # For 3 classes (N, P, Y)

    # Compile the model
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Create and train the model
input_dim = X_train.shape[1]
fnn_model = create_fnn(input_dim)

# Train the model
fnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Predict the classes
y_pred_fnn = fnn_model.predict(X_test)
y_pred_fnn = np.argmax(y_pred_fnn, axis=1)  # Get the class with the highest probability

# Performance Metrics for FNN
accuracy_fnn = accuracy_score(y_test, y_pred_fnn)
precision_fnn = precision_score(y_test, y_pred_fnn, average='weighted')
recall_fnn = recall_score(y_test, y_pred_fnn, average='weighted')
roc_auc_fnn = roc_auc_score(y_test, fnn_model.predict(X_test), multi_class='ovr', average='weighted')
conf_matrix_fnn = confusion_matrix(y_test, y_pred_fnn)

# Print metrics
print("FNN Accuracy:", accuracy_fnn)
print("FNN Precision:", precision_fnn)
print("FNN Recall:", recall_fnn)
print("FNN AUC-ROC:", roc_auc_fnn)
print("FNN Confusion Matrix:\n", conf_matrix_fnn)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.3001 - loss: 1.0758 - val_accuracy: 0.8650 - val_loss: 0.7265
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8180 - loss: 0.6641 - val_accuracy: 0.8650 - val_loss: 0.4097
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8558 - loss: 0.4166 - val_accuracy: 0.8650 - val_loss: 0.3615
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8547 - loss: 0.3930 - val_accuracy: 0.8650 - val_loss: 0.3358
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8444 - loss: 0.3812 - val_accuracy: 0.8650 - val_loss: 0.3021
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8447 - loss: 0.3525 - val_accuracy: 0.8700 - val_loss: 0.2799
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8497 - loss: 0.3362 - val_accuracy: 0.8800 - val_loss: 0.2548
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8618 - loss: 0.3196 - val_accuracy: 0.9050 - val_loss: 0.2429
Epo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [68]:
import brian2 as b2
import numpy as np

# Define parameters
n_neurons = 100  # Number of neurons in the group
duration = 1 * b2.second  # Duration of the simulation

# Define the neuron model
eqs = '''
dv/dt = (I - v) / (10 * ms) : 1  # Membrane potential equation
I : 1  # External input current
'''

# Create the neuron group
neurons = b2.NeuronGroup(n_neurons, eqs, threshold='v > 1', reset='v = 0', method='euler')

# Create spike monitor
spike_monitor = b2.SpikeMonitor(neurons)

# Example of encoding input data into spikes (rate coding)
# Assuming diabetes dataset has numerical features 'AGE', 'BMI', etc.
age_feature = 50  # Example: encoding age feature as 50
encoded_input = age_feature  # Map this to a spike rate, e.g., age = 50 means 50 spikes per second

# Set input current based on the feature values
neurons.I = encoded_input

# Run the network simulation
b2.run(duration)

# Collect spikes for analysis
spike_times = spike_monitor.spike_trains()

# Analyze the spike times to extract information for classification
print(f"Spike times for neurons: {spike_times}")

WARNING    'statemonitor' has been included in the network but not the object on which it depends.Setting 'statemonitor' to inactive. [brian2.core.magic.dependency_warning]


Spike times for neurons: {0: array([2.000e-01, 5.000e-01, 8.000e-01, ..., 9.992e+02, 9.995e+02,
       9.998e+02]) * msecond, 1: array([2.000e-01, 5.000e-01, 8.000e-01, ..., 9.992e+02, 9.995e+02,
       9.998e+02]) * msecond, 2: array([2.000e-01, 5.000e-01, 8.000e-01, ..., 9.992e+02, 9.995e+02,
       9.998e+02]) * msecond, 3: array([2.000e-01, 5.000e-01, 8.000e-01, ..., 9.992e+02, 9.995e+02,
       9.998e+02]) * msecond, 4: array([2.000e-01, 5.000e-01, 8.000e-01, ..., 9.992e+02, 9.995e+02,
       9.998e+02]) * msecond, 5: array([2.000e-01, 5.000e-01, 8.000e-01, ..., 9.992e+02, 9.995e+02,
       9.998e+02]) * msecond, 6: array([2.000e-01, 5.000e-01, 8.000e-01, ..., 9.992e+02, 9.995e+02,
       9.998e+02]) * msecond, 7: array([2.000e-01, 5.000e-01, 8.000e-01, ..., 9.992e+02, 9.995e+02,
       9.998e+02]) * msecond, 8: array([2.000e-01, 5.000e-01, 8.000e-01, ..., 9.992e+02, 9.995e+02,
       9.998e+02]) * msecond, 9: array([2.000e-01, 5.000e-01, 8.000e-01, ..., 9.992e+02, 9.995e+02,
    